In [1]:
import pandas as pd
import ast


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from imdb import Cinemagoer

KeyboardInterrupt: 

In [ ]:
imdb = Cinemagoer()

In [ ]:
ratings = pd.read_csv("./data/ratings.csv")
movies = pd.read_csv("./data/movies.csv")
links = pd.read_csv("./data/links.csv", index_col = "movieId")

In [ ]:
def get_directors(movie_id):
    theMovie = imdb.get_movie(movie_id)
    director_name = [director['name'] for director in theMovie['directors']]
    return director_name

In [ ]:
tmdv_credits = pd.read_csv("../Project/rec_app/data/tmdb_5000_credits.csv")
tmdv_movies = pd.read_csv("../Project/rec_app/data/tmdb_5000_movies.csv")

In [ ]:
tmdv_credits.sample(2)

,movie_id,title,cast,crew
2432,9310,American Dreamz,"[{""cast_id"": 1, ""character"": ""Martin Tweed"", ""...","[{""credit_id"": ""53c4f738c3a3682479001259"", ""de..."
3197,8883,Flame & Citron,"[{""cast_id"": 28, ""character"": ""Flame"", ""credit...","[{""credit_id"": ""52fe44c4c3a36847f80a8e85"", ""de..."


In [ ]:
tmdv_movies.sample(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
2340,18000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 18, ""nam...",http://www.soulsurferthemovie.com/,43959,"[{""id"": 271, ""name"": ""competition""}, {""id"": 81...",en,Soul Surfer,Soul Surfer is the inspiring true story of tee...,19.234672,"[{""name"": ""Mandalay Pictures"", ""id"": 551}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2011-04-08,47088990,106.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"When you come back from a loss, beat the odds,...",Soul Surfer,6.9,342
455,75000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 18, ""nam...",NaN,8834,"[{""id"": 242, ""name"": ""new york""}, {""id"": 591, ...",en,Conspiracy Theory,A man obsessed with conspiracy theories become...,22.179658,"[{""name"": ""Silver Pictures"", ""id"": 1885}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1997-08-07,136982834,135.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,What if your most paranoid nightmares had just...,Conspiracy Theory,6.5,431


In [ ]:
def convert(text):
    keyword_list = []
    for i in ast.literal_eval(text):
        keyword_list.append(i['name']) 
    return keyword_list

In [ ]:
def get_director(text):
    directors = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            directors.append(i['name'])
    return directors

In [ ]:
tmdv_credits['director'] = tmdv_credits['crew'].apply(get_director)

In [ ]:
tmdv_movies = tmdv_movies[['genres', 'id', 'keywords', 'production_companies', 'title', 'vote_average']]

In [ ]:
tmdv_movies['genres'] = tmdv_movies['genres'].apply(convert)
tmdv_movies['keywords'] = tmdv_movies['keywords'].apply(convert)
tmdv_movies['production_companies'] = tmdv_movies['production_companies'].apply(convert)

1. combine features

In [ ]:
tmdv_movies['comb_feat'] = tmdv_movies['genres'] + tmdv_movies['keywords'] + tmdv_movies['production_companies']

In [ ]:
tmdv_movies.sample(2)

,genres,id,keywords,production_companies,title,vote_average,comb_feat
687,"[Action, Adventure, War]",1271,"[evisceration, javelin, shield, army, fall fro...","[Virtual Studios, Legendary Pictures, Hollywoo...",300,7.0,"[Action, Adventure, War, evisceration, javelin..."
1123,"[Comedy, Romance]",11066,"[role of women, ladykiller, success, ladies' m...",[Paramount Pictures],Boomerang,5.4,"[Comedy, Romance, role of women, ladykiller, s..."


In [ ]:
tmdv_movies['comb_feat'] = tmdv_movies['comb_feat'].apply(lambda x: ' '.join(x))

In [ ]:
tmdv_movies.sample(2)

,genres,id,keywords,production_companies,title,vote_average,comb_feat
2358,"[Drama, Thriller]",2575,"[dancing, spy, birthday, map, fireworks, bridg...","[Columbia Pictures, Merlin Films]",The Tailor of Panama,6.2,Drama Thriller dancing spy birthday map firewo...
828,"[Action, Crime]",24,"[japan, coma, martial arts, kung fu, underworl...","[Miramax Films, A Band Apart, Super Cool ManChu]",Kill Bill: Vol. 1,7.7,Action Crime japan coma martial arts kung fu u...


In [ ]:
tmdv_movies['comb_feat'] = tmdv_movies['comb_feat'] + " " + tmdv_movies['vote_average'].astype(str)

In [ ]:
reco_df = tmdv_movies[['id','title','comb_feat']]

In [ ]:
reco_df

,id,title,comb_feat
0,19995,Avatar,Action Adventure Fantasy Science Fiction cultu...
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drug abuse exot...
2,206647,Spectre,Action Adventure Crime spy based on novel secr...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dc comics crime fi...
4,49529,John Carter,Action Adventure Science Fiction based on nove...
...,...,...,...
4798,9367,El Mariachi,Action Crime Thriller united states–mexico bar...
4799,72766,Newlyweds,Comedy Romance 5.9
4800,231617,"Signed, Sealed, Delivered",Comedy Drama Romance TV Movie date love at fir...
4801,126186,Shanghai Calling,5.7


In [ ]:
# reco_df.to_csv('reco_mat.csv',index=False)

In [ ]:
def recommend_neighbour_movie(movie_title):
    """
    Runs model and gets movies simimlar to original title
    """
    cv = CountVectorizer(max_features=5000)
    count_matrix = cv.fit_transform(reco_df["comb_feat"]).toarray()
    similarity = cosine_similarity(count_matrix)

    index = reco_df[reco_df['title'] == movie_title].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(reco_df.iloc[i[0]].title)